In [1]:
import pandas as pd

df = pd.read_csv('D1.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19663 entries, 0 to 19662
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    19663 non-null  int64  
 1   StockCode    19663 non-null  object 
 2   Description  19075 non-null  object 
 3   Quantity     19663 non-null  int64  
 4   InvoiceDate  19663 non-null  object 
 5   UnitPrice    19663 non-null  float64
 6   CustomerID   15678 non-null  float64
 7   Country      19663 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 1.2+ MB


In [2]:
df2 = df.dropna(subset=['Description']).copy()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19075 entries, 0 to 19662
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    19075 non-null  int64  
 1   StockCode    19075 non-null  object 
 2   Description  19075 non-null  object 
 3   Quantity     19075 non-null  int64  
 4   InvoiceDate  19075 non-null  object 
 5   UnitPrice    19075 non-null  float64
 6   CustomerID   15678 non-null  float64
 7   Country      19075 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 1.3+ MB


In [3]:
df2['StockCode'].unique()

array(['85123A', '84879', '22386', '20725', '21212', '22457', '22469',
       '85099B', '20727', '22423', '47566', '85123a', '22720'],
      dtype=object)

In [4]:
df2['Description'].unique()

array(['WHITE HANGING HEART T-LIGHT HOLDER',
       'ASSORTED COLOUR BIRD ORNAMENT', 'JUMBO BAG PINK POLKADOT',
       'LUNCH BAG RED RETROSPOT', 'PACK OF 72 RETROSPOT CAKE CASES',
       'NATURAL SLATE HEART CHALKBOARD ', 'HEART OF WICKER SMALL',
       'JUMBO BAG RED RETROSPOT', 'LUNCH BAG  BLACK SKULL.',
       'REGENCY CAKESTAND 3 TIER', 'PARTY BUNTING',
       'SET OF 3 CAKE TINS PANTRY DESIGN '], dtype=object)

In [5]:
# number of StockCode is much larger than the Description, so it seems useless
# for the same item, the stockcode will change
# need to transform categorical values to numerical values for use

In [6]:
transactions = df2.groupby('InvoiceNo')['Description'].apply(list)
print(transactions.head(10))

InvoiceNo
536365                 [WHITE HANGING HEART T-LIGHT HOLDER]
536367                      [ASSORTED COLOUR BIRD ORNAMENT]
536373                 [WHITE HANGING HEART T-LIGHT HOLDER]
536375                 [WHITE HANGING HEART T-LIGHT HOLDER]
536378    [JUMBO BAG PINK POLKADOT, LUNCH BAG RED RETROS...
536384    [NATURAL SLATE HEART CHALKBOARD , HEART OF WIC...
536386                            [JUMBO BAG RED RETROSPOT]
536388                              [HEART OF WICKER SMALL]
536390    [WHITE HANGING HEART T-LIGHT HOLDER, JUMBO BAG...
536392                      [ASSORTED COLOUR BIRD ORNAMENT]
Name: Description, dtype: object


In [7]:
print(transactions.info())

<class 'pandas.core.series.Series'>
Index: 9568 entries, 536365 to 581585
Series name: Description
Non-Null Count  Dtype 
--------------  ----- 
9568 non-null   object
dtypes: object(1)
memory usage: 149.5+ KB
None


In [8]:
# Q: What pre-processing was required on the dataset before building the association mining model? 
# A: First check missing values. "CustomerID" and "Description" have missing values. "CustomerID" is not used so that's okay; however, samples with 
#    missing values from "Descrition" should be removed. 
#    Then find out what variables represent unique ID for products. I examined "StockCode" and "Description" and found that "Descriptions" with 13 unique values 
#    represents the unique ID for products.  

# Q: What variables did you include in the analysis? Justify your choice.
# A: The "Descriptions" and "InvoiceNo" will be used in the analysis. Becasue "Descriptions" is unique for purchased products and "InvoiceNo" is unique for each
#    purchase. Using this two variables, we can know what products were bought each time and hence can find associted items/products. 

In [9]:
# Note that we have 13 items and the support level should be small to capture meaningful results. 1/13=0.076 (suppose one item was purchase, and the
# chance of it was purchase is 0.076). So I choose 0.02 as the minimum support. The typical confidence level is usually between 0.5-0.9 and I choose
# 0.5, so that means when item A is purhcased, at least 50% chance that it will be followed by B purchased

from apyori import apriori
# type cast the transactions from pandas into normal list format and run apriori
transaction_list = list(transactions)
results = list(apriori(transaction_list, min_support=0.02, confidence=0.5))
# print first 5 rules
print(results[:5])

[RelationRecord(items=frozenset({'ASSORTED COLOUR BIRD ORNAMENT'}), support=0.15206939799331104, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'ASSORTED COLOUR BIRD ORNAMENT'}), confidence=0.15206939799331104, lift=1.0)]), RelationRecord(items=frozenset({'HEART OF WICKER SMALL'}), support=0.12552257525083613, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'HEART OF WICKER SMALL'}), confidence=0.12552257525083613, lift=1.0)]), RelationRecord(items=frozenset({'JUMBO BAG PINK POLKADOT'}), support=0.12729933110367894, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'JUMBO BAG PINK POLKADOT'}), confidence=0.12729933110367894, lift=1.0)]), RelationRecord(items=frozenset({'JUMBO BAG RED RETROSPOT'}), support=0.21864548494983277, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'JUMBO BAG RED RETROSPOT'}), confidence=0.21864548494983277, lift=1.0)]), Relatio

In [10]:
def convert_apriori_results_to_pandas_df(results):
    rules = []
    for rule_set in results:
        for rule in rule_set.ordered_statistics:
# items_base = left side of rules, items_add = right side
# support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
                rule_set.support, rule.confidence, rule.lift])
# typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 'Confidence', 'Lift'])
result_df = convert_apriori_results_to_pandas_df(results)
print(result_df.info())
print(result_df.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Left_side   173 non-null    object 
 1   Right_side  173 non-null    object 
 2   Support     173 non-null    float64
 3   Confidence  173 non-null    float64
 4   Lift        173 non-null    float64
dtypes: float64(3), object(2)
memory usage: 6.9+ KB
None
                        Left_side  \
0                                   
1                                   
2                                   
3                                   
4                                   
5                                   
6                                   
7                                   
8                                   
9                                   
10                                  
11                                  
12                                  
13  ASSORTED COLOUR BIRD ORNAMENT 

In [11]:
specific_value = 'LUNCH BAG RED RETROSPOT'
filtered_df = result_df[result_df['Left_side'] == specific_value]
print(filtered_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 26 to 169
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Left_side   12 non-null     object 
 1   Right_side  12 non-null     object 
 2   Support     12 non-null     float64
 3   Confidence  12 non-null     float64
 4   Lift        12 non-null     float64
dtypes: float64(3), object(2)
memory usage: 576.0+ bytes
None


In [12]:
filtered_df = filtered_df.sort_values(by='Lift', ascending=False)
print(filtered_df)

                   Left_side                                       Right_side  \
169  LUNCH BAG RED RETROSPOT  LUNCH BAG  BLACK SKULL.,JUMBO BAG RED RETROSPOT   
83   LUNCH BAG RED RETROSPOT                          LUNCH BAG  BLACK SKULL.   
162  LUNCH BAG RED RETROSPOT  JUMBO BAG RED RETROSPOT,JUMBO BAG PINK POLKADOT   
62   LUNCH BAG RED RETROSPOT                          JUMBO BAG RED RETROSPOT   
50   LUNCH BAG RED RETROSPOT                          JUMBO BAG PINK POLKADOT   
100  LUNCH BAG RED RETROSPOT                  PACK OF 72 RETROSPOT CAKE CASES   
103  LUNCH BAG RED RETROSPOT                                    PARTY BUNTING   
97   LUNCH BAG RED RETROSPOT                  NATURAL SLATE HEART CHALKBOARD    
26   LUNCH BAG RED RETROSPOT                            HEART OF WICKER SMALL   
109  LUNCH BAG RED RETROSPOT                SET OF 3 CAKE TINS PANTRY DESIGN    
112  LUNCH BAG RED RETROSPOT               WHITE HANGING HEART T-LIGHT HOLDER   
106  LUNCH BAG RED RETROSPOT

In [13]:
# Q: Identify top-5 common items that customers have purchased along with ‘LUNCHBAG RED RETROSPOT’.
# A: Top 5 common items that customers have purchased along with 'LUNCHBAG RED RETROSPOT' are 'JUMBO BAG RED RETROSPOT', 'LUNCH BAG BLACK SKULL'
# 'JUMBO BAG PINK POLKADOT', 'PACK OF 72 RETROSPOT CAKE CASES', AND 'PARTY BUNTING'

In [14]:
# Q: Can you perform sequence analysis on this dataset? If yes, present your results. Ifnot, rationalise why.
# A: Yes, as we have CustomerID and InvoiceDate in the dateset. 